
# <span style='color:midnightblue'> Python para Actuarios Vida </span>

![ADRISK](Imagenes/KN.png)


## <span style='color:midnightblue'> Sesión 04 :Renta vitalicias y   seguros de vida (Parte 01)  </span>

## Anualidad versus renta vitalicia: ¡tenga en cuenta la diferencia!

- Una anualidad cierta es el préstamo de un automóvil
- Una renta vitalicia es un plan de retiro

**La anualidad (cierta)** ofrece una serie de pagos garantizados.   
![](imagenes/cierto.png)
**La renta vitalicia** depende de la **supervivencia** del receptor.
![](imagenes/renta1.png)

## Dotación pura

El producto se vende a **(x)** en el momento 0. 
![](imagenes/dp1.png)

#### VPE de una dotación pura
Valor Presente Esperado:
![](imagenes/dp2.png)

El VPE es : 
- $_{k}E_{x}=1*v(k)*_{k}p_{x}$

Con una **anualidad cierta**,  el beneficio de 1 en el momento k está garantizado   
el VP es :
- $v(k)$

In [1]:
i = 0.03
k = 5
discount_factor = (1+i)**-k
pv = 1*discount_factor
pv

0.8626087843841639

In [2]:
import pandas as pd
import numpy as np
life_table = pd.read_excel("life_table.xlsx",engine='openpyxl') 
life_table

,age,qx,lx,ex,dx
0,0,0.00382,100000,77.95,382
1,1,0.00047,99618,77.25,47
2,2,0.00019,99572,76.28,19
3,3,0.00015,99552,75.30,15
4,4,0.00013,99538,74.31,13
...,...,...,...,...,...
106,106,0.52076,15,1.38,8
107,107,0.53591,7,1.33,4
108,108,0.54984,3,1.29,2
109,109,0.56257,2,1.26,1


Con **dotación pura**, el beneficio de 1 EUR en el momento K no está garantizado

In [3]:
qx = life_table.qx
px = 1- qx

kpx = np.prod(px[65:69+1])            
kpx         


0.9145881430551792

In [4]:
1*discount_factor*kpx

0.7889317662929979

####  Caso 01 :
- PV de pago garantizado de 10,000 en 5 años
- EPV de dotación pura de 10,000 en 5 años para (20)


In [5]:
# pago garantizado
k = 5
i = 0.02
b = 10000
PV =  b * (1+i)**-k
PV

9057.308098299158

In [13]:
# Probabilidad de supervivencia a 5 años de (20)
kpx = np.prod(px[20:25])            
kpx    

0.9968539604119112

In [7]:
# EPV de dotación pura de 10,000 en 5 años para (20)
PV*kpx

9028.81344846039

####  Caso 02 :

- PV de pagos garantizados de 10,000 en 5, 10 y 30 años
- EPV de dotaciones puras de 10,000 en 5, 10 y 30 años para (20)

In [8]:
# pago garantizado
k = pd.Series([5,10,30])
i = 0.02
b = 10000
PV =  b * (1+i)**-k
PV

0    9057.308098
1    8203.482999
2    5520.708890
dtype: float64

In [26]:
# Probabilidades de supervivencia de (20) 
kpx = np.cumprod(px[20:len(px)-1])             
kpx.head()

20    0.999320
21    0.998650
22    0.997991
23    0.997422
24    0.996854
Name: qx, dtype: float64

In [27]:
# eliminamos el indice para facililar el filtro
kpx.reset_index(inplace=True,drop = True)
kpx

0     0.999320
1     0.998650
2     0.997991
3     0.997422
4     0.996854
        ...   
85    0.000153
86    0.000073
87    0.000034
88    0.000015
89    0.000007
Name: qx, Length: 90, dtype: float64

In [28]:
kpx[[4,9,29]]


4     0.996854
9     0.993062
29    0.962788
Name: qx, dtype: float64

In [29]:
PV*kpx[[4,9,29]].values


0    9028.813448
1    8146.564800
2    5315.272884
dtype: float64

In [32]:
# Dado que los peridos están definidos en la serie k , los usamos para hacer un filtro más rapido
b*(1+i)**-k*kpx[k-1].values


0    9028.813448
1    8146.564800
2    5315.272884
dtype: float64

In [34]:
# En caso de no resetear el index :
# Probabilidades de supervivencia de (20) 
kpx = np.cumprod(px[20:len(px)-1])             
kpx
#agregamos la edad al filtro de probabildiades de sobrevivencia
PV*kpx[ 20 + k-1].values

0    9028.813448
1    8146.564800
2    5315.272884
dtype: float64

## La renta vitalicia total, temporal y diferida

#### Una serie de beneficios


![](imagenes/dp2.png)

¿Y si?   
- ¿El beneficio es $c_{k}$ en lugar de 1 ?
- ¿Una serie de dotaciones puras en lugar de solo una?

###  Estructura general

Una renta vitalicia en $(x)$ con vector de beneficios:   

- $(c_{0}, c_{1},…,c_{k},…)$   

Secuencia de dotaciones puras:    
- cada una con $c_{k}⋅v(k)⋅_{k}p_{x}$ como valor presente esperado (EPV)   

juntos :   

-  $\sum_{k=0}^{\infty} c_{k}⋅v(k)⋅_{k}p_{x}$ es el EPV.

### Rentas vitalicias en Python
![](imagenes/rv1.png)

In [35]:
i = 0.03
benefits = pd.Series([500,400,300]+ [200]*5)
benefits


0    500
1    400
2    300
3    200
4    200
5    200
6    200
7    200
dtype: int64

In [38]:
per = len(benefits)
per
exp = pd.Series(range(per))  #range(8) 0 hasta 7 , range(0,8), range(0,8,2)
exp

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
dtype: int64

In [39]:
discount_factor =(1+i)**-exp
discount_factor

0    1.000000
1    0.970874
2    0.942596
3    0.915142
4    0.888487
5    0.862609
6    0.837484
7    0.813092
dtype: float64

In [40]:
kpx = np.cumprod(px[65:72])             
kpx

65    0.984830
66    0.968364
67    0.951960
68    0.933682
69    0.914588
70    0.893681
71    0.871339
Name: qx, dtype: float64

In [41]:
# ignore_index= True para resetear le indice al concatenar
kpx = pd.concat( [pd.Series(1), np.cumprod(px[65:72])  ] ,ignore_index= True )
kpx

0    1.000000
1    0.984830
2    0.968364
3    0.951960
4    0.933682
5    0.914588
6    0.893681
7    0.871339
dtype: float64

In [44]:
benefits *discount_factor*kpx.values
# en caso no resetee el indice :
# usar la siguiente formula : benefits * discount_factor**kpx.values

0    500.000000
1    382.458252
2    273.832682
3    174.235571
4    165.912862
5    157.786353
6    149.688696
7    141.695611
dtype: float64

In [45]:
sum(benefits *discount_factor*kpx.values)

1945.6100275670124

### Renta vitalicia adelantada (Annuity Due)

Rentas vitalicia **adelantada**: paga $c_{k}$ al comienzo del año $(k + 1)$.
![](imagenes/rv2.png)
### Renta vitalicia vencida (Annuity Immediate)
Renta vitalicia **inmediata**: paga $c_{k}$ al final del año $(k + 1)$.
![](imagenes/rv3.png)

### Caso 03:
Calcular $ä_{35}$ (adelantada) para una tasa de interés constante i = 3%


In [47]:
#renta vitalicia adelantada de (35)

kpx = pd.concat( [pd.Series(1), np.cumprod(px[35: ])] ,ignore_index= True )
# alternativa : kpx = pd.concat( [pd.Series(1), np.cumprod(px[35:len(px) ])] ,ignore_index= True )
kpx

0     1.000000
1     0.998840
2     0.997801
3     0.996823
4     0.995617
        ...   
72    0.000074
73    0.000034
74    0.000015
75    0.000007
76    0.000000
Length: 77, dtype: float64

In [49]:
i = 0.03
per = len(kpx)
exp = pd.Series(range(per))
exp

discount_factor =  (1 + i)**-(exp)
discount_factor

0     1.000000
1     0.970874
2     0.942596
3     0.915142
4     0.888487
        ...   
72    0.119047
73    0.115580
74    0.112214
75    0.108945
76    0.105772
Length: 77, dtype: float64

In [57]:
benefits= pd.Series([1]*per)
benefits

0     1
1     1
2     1
3     1
4     1
     ..
72    1
73    1
74    1
75    1
76    1
Length: 77, dtype: int64

In [58]:
benefits*discount_factor*kpx

0     1.000000e+00
1     9.697476e-01
2     9.405233e-01
3     9.122346e-01
4     8.845930e-01
          ...     
72    8.795915e-06
73    3.963200e-06
74    1.732111e-06
75    7.356090e-07
76    0.000000e+00
Length: 77, dtype: float64

In [59]:
sum(benefits*discount_factor*kpx) 

24.443116876690283

### Caso 04:
Calcular $a_{35}$ (inmediata) para una tasa de interés constante i = 3%

In [60]:
kpx = np.cumprod(px[35: ])
kpx.reset_index(inplace=True,drop = True)
kpx

0     0.998840
1     0.997801
2     0.996823
3     0.995617
4     0.994482
        ...   
71    0.000074
72    0.000034
73    0.000015
74    0.000007
75    0.000000
Name: qx, Length: 76, dtype: float64

In [62]:
i = 0.03
per = len(kpx)
exp = pd.Series(range(1,per+1))
exp

discount_factor =  (1 + i)**-(exp)
discount_factor

0     0.970874
1     0.942596
2     0.915142
3     0.888487
4     0.862609
        ...   
71    0.119047
72    0.115580
73    0.112214
74    0.108945
75    0.105772
Length: 76, dtype: float64

In [63]:
benefits= pd.Series([1]*per)
benefits

0     1
1     1
2     1
3     1
4     1
     ..
71    1
72    1
73    1
74    1
75    1
Length: 76, dtype: int64

In [64]:
 benefits*discount_factor*kpx

0     9.697476e-01
1     9.405233e-01
2     9.122346e-01
3     8.845930e-01
4     8.578491e-01
          ...     
71    8.795915e-06
72    3.963200e-06
73    1.732111e-06
74    7.356090e-07
75    0.000000e+00
Length: 76, dtype: float64

In [65]:
sum(benefits*discount_factor*kpx) 

23.443116876690286

### Renta temporal adelantada

Anualidad **adelantada** temporar: máximo de $n$ años, en el momento 0 hasta $n - 1$.   
![](imagenes/temp.png)


### Rentas vitalicias diferidas vencidas

Anualidad **diferida**: apartir del momento  $u$.   
![](imagenes/difer.png)


###  Caso 05:

Construya una función para calcular el EPV de una renta vitalicia  adelantada para una determinada **edad**, **i** y **tabla de mortalidad** 
- EPV de una renta vitalicia completa adelantada de (20) a i = 2% usando life_table
- EPV de una renta vitalicia completa adelantada de (20) a i = 5% usando life_table
- EPV de una renta vitalicia completa adelantada de (65) a i = 2% usando life_table

In [67]:
def life_annuity_due(age,i,life_table):
    
    px = 1- life_table.qx
    
    kpx = pd.concat( [pd.Series(1), np.cumprod(px[age:  ])] ,ignore_index= True )
    per = len(kpx)
    exp = pd.Series(range(per))
    discount_factors =  (1 + i)**-(exp)
    return sum(discount_factors * kpx)    

In [68]:
#- EPV de una renta vitalicia completa adelantada de (20) a i = 2% usando life_table
life_annuity_due(20, 0.02 , life_table)

34.52303917635918

In [69]:
# - EPV de una renta vitalicia completa adelantada de (20) a i = 5% usando life_table
life_annuity_due(20, 0.05 , life_table)

19.444789837497627

In [70]:
# EPV de una renta vitalicia completa adelantada de (65) a i = 2% usando life_table
life_annuity_due(65, 0.02 , life_table)

14.896607058095476

###  Caso 06:

Construya una función para calcular el EPV de una renta vitalicia inmediata para una determinada **edad**, **i** y **tabla de mortalidad** 
- EPV de una renta vitalicia completa inmediata de (20) a i = 2% usando life_table
- EPV de una renta vitalicia completa inmediata de (20) a i = 5% usando life_table
- EPV de una renta vitalicia completa inmediata de (65) a i = 2% usando life_table

In [71]:
def life_inmediate(age,i,life_table):
    px = 1- life_table.qx
    kpx = np.cumprod(px[age:  ])
    kpx.reset_index(inplace=True,drop = True)
    per = len(kpx)
    exp = pd.Series(range(1,per+1))
    discount_factors =  (1 + i)**-(exp)
    return sum(discount_factors * kpx)    

In [72]:
life_inmediate(20, 0.02 , life_table)

33.52303917635917

In [73]:
life_inmediate(20, 0.05 , life_table)

18.444789837497627

In [74]:
life_inmediate(65, 0.02 , life_table)

13.896607058095478

In [75]:
life_annuity_due(20, 0.02 , life_table)- life_inmediate(20, 0.02 , life_table)

1.000000000000007

In [76]:
life_annuity_due(20, 0.05 , life_table)-life_inmediate(20, 0.05 , life_table)

1.0

In [77]:
life_annuity_due(65, 0.02 , life_table)-life_inmediate(65, 0.02 , life_table)

0.9999999999999982

###  Caso 07:

Construya una función para calcular el EPV de una renta temporal adelantada para una determinada **edad**, periodo **n** años , **i** y **tabla de mortalidad** 
- EPV de una renta renta temporal adelantada de (20) a i = 2% , n = 10 , usando life_table
- EPV de una renta renta temporal adelantada de (20) a i = 5%, n = 5, usando life_table
- EPV de una renta  renta temporal adelantada de (65) a i = 2%, n = 10,usando life_table

In [82]:
def temporary_life(age,i,n,life_table):
    px = 1- life_table.qx
    kpx = pd.concat([pd.Series(1), np.cumprod(px[age:age+n-1])] ,ignore_index= True )
    per = len(kpx)
    exp = pd.Series(range(0, per))
    discount_factors =  (1 + i)**-(exp)
    return sum(discount_factors * kpx)    

In [83]:
temporary_life(20,0.02,10, life_table)  

9.136235726105037

In [84]:
temporary_life(20,0.05,5, life_table)  

4.54022313513006

In [86]:
temporary_life(65,0.02,10, life_table)  

8.447955099490208

In [87]:
def temporary_life_in(age,i,n,life_table):
    px = 1- life_table.qx
    kpx = np.cumprod(px[age:age+n])
    kpx.reset_index(inplace=True,drop = True)
    per = len(kpx)
    exp = pd.Series(range(1, per+1))
    discount_factors =  (1 + i)**-(exp)
    return sum(discount_factors * kpx)   

In [88]:
temporary_life_in(65,0.02,10, life_table)

8.100194165859358

In [89]:
temporary_life(65,0.02,10, life_table)  - temporary_life_in(65,0.02,10, life_table)

0.34776093363085003

In [90]:
# la difencia es el 1 - el ultimo pago(en valor presente) de la renta inmediate
1- np.prod(px[65:65+10])*(1+0.02)**-10

0.34776093363085114

### Renta Garantizada

Flexibilidad adicional: rentas vitalicias con plazo garantizado.
Un contrato típico:
- **Inicialmente:**   
    -  los beneficios se pagan independientemente de si el beneficiario está vivo o no.
- **Después:**   
    -  beneficios se pagan condicionados a la supervivencia.

###  Caso 08:

Mr. X tiene 35 años.
La aseguradora "XYZ" le ha regalado un premio especial: una renta vitalicia de 10.000 PEN
cada año de por vida!
El primer pago comienza al inicio del primer año,
Además, **los primeros 10 pagos están garantizados.**

¿Puedes calcular el valor de su premio?
![](imagenes/gua.png)

In [107]:
# Survival probabilities of (35)
age = 35
kpx = pd.concat([pd.Series(1), np.cumprod(px[age:])] ,ignore_index= True )
kpx

0     1.000000
1     0.998840
2     0.997801
3     0.996823
4     0.995617
        ...   
72    0.000074
73    0.000034
74    0.000015
75    0.000007
76    0.000000
Length: 77, dtype: float64

In [108]:
# Discount factors
i = 0.03
per = len(kpx)
exp = pd.Series(range(per))
discount_factor =  (1 + i)**-(exp)
discount_factor 

0     1.000000
1     0.970874
2     0.942596
3     0.915142
4     0.888487
        ...   
72    0.119047
73    0.115580
74    0.112214
75    0.108945
76    0.105772
Length: 77, dtype: float64

In [109]:
benefits_guaranteed = pd.Series([0] +[10000]*10 + [0]*(len(kpx)-11))
benefits_nonguaranteed = pd.Series([0] + [0]*10   + [10000]*(len(kpx)-11))                                


In [110]:
 #PV of the guaranteed annuity
sum(benefits_guaranteed*discount_factor) 

85302.02836775829

In [111]:
#EPV of the guaranteed annuity              
sum(benefits_nonguaranteed*discount_factor*kpx)

149682.4775080444

In [112]:
 #PV of the guaranteed annuity   +  #EPV of the guaranteed annuity          
sum(benefits_guaranteed*discount_factor) + sum(benefits_nonguaranteed*discount_factor*kpx)     

234984.5058758027

### Cálculo de primas y planes de retiro
#### El pago primas

- Objetivo del cálculo de la prima:
     * Las primas + los ingresos por intereses deben coincidir con los beneficios
- Solución:
    * Establecer la equivalencia actuarial entre el vector premium y los beneficios
    * Trate los pagos de primas como una renta vitalicia en $(x)$.

### Caso 09: 
Mrs. X tiene 35 años.
Quiere comprar una **renta vitalicia** que proporcione 12.000 PEN  anuales de por vida, a partir de los 65 años.   
El financiará este producto con una prima anual pagadera a 30 años a partir de la edad 35.    
Las primas se reducen a la mitad después de 15 años.  
**¿Cuál es su prima inicial?**
![](imagenes/planr.png)

In [113]:
# Survival probabilities of (35)
age = 35
kpx = pd.concat([pd.Series(1), np.cumprod(px[age:])] ,ignore_index= True )
kpx

0     1.000000
1     0.998840
2     0.997801
3     0.996823
4     0.995617
        ...   
72    0.000074
73    0.000034
74    0.000015
75    0.000007
76    0.000000
Length: 77, dtype: float64

In [114]:
# Discount factors
i = 0.03
per = len(kpx)
exp = pd.Series(range(per))
discount_factors =  (1 + i)**-(exp)
discount_factors 

0     1.000000
1     0.970874
2     0.942596
3     0.915142
4     0.888487
        ...   
72    0.119047
73    0.115580
74    0.112214
75    0.108945
76    0.105772
Length: 77, dtype: float64

In [115]:
benefits = pd.Series([0]*30 +[12000]*(len(kpx)-30))
sum(benefits*discount_factors*kpx)

58259.06521761427

In [116]:
# Patrón de primas
rho = pd.Series([1]*15 +[0.5]*15+[0]*(len(kpx)-30))
sum(rho*discount_factors*kpx)   

15.887389072699465

In [117]:
# Entonces. la prima es  EPV(benefits)/EPV(rho)


premium_payments = sum(benefits*discount_factors*kpx) /sum(rho*discount_factors*kpx)   
premium_payments

3667.0005972048198

### Equivalencia Actuarial

$P = \frac{EPV(benefits)}{EPV(rho)}$    
    
    
$P*{EPV(rho)} = {EPV(benefits)}$

### Caso 10: 
Mrs. X tiene 40 años.
Quiere comprar una **renta vitalicia** que proporcione 18.000 PEN  anuales de por vida, a partir de los 65 años.      
Las prima se empieza a pagar desde hoy  se reducen a la mitad después de 15 años.  
**¿Cuál es su prima inicial?**    
**De ser prima unica, ¿Cuál es el monto?**   
**¿Cuál es su esperanza de vida reducida?**   
**¿Calcule el EPV de los beneficios si vive hasta los 75?**   
**¿Calcule el EPV de los beneficios si vive hasta los 95?**



In [118]:
age = 40
kpx = pd.concat([pd.Series(1),np.cumprod(px[age:])], ignore_index = True)
kpx

0     1.000000
1     0.998250
2     0.996942
3     0.995447
4     0.993436
        ...   
67    0.000074
68    0.000034
69    0.000016
70    0.000007
71    0.000000
Length: 72, dtype: float64

In [119]:
i = 0.03
per = len(kpx)
exp =pd.Series(range(per))
discount_factors = (1+i)**-exp

In [120]:
benefits = pd.Series([0]*25 + [18000]*(len(kpx)-25))
benefits

0         0
1         0
2         0
3         0
4         0
      ...  
67    18000
68    18000
69    18000
70    18000
71    18000
Length: 72, dtype: int64

In [121]:
# prima unica
prima_unica = sum(benefits*discount_factors*kpx)
prima_unica

101869.43085899291

In [122]:
#patron de primas
rho = pd.Series([1]*15 + [0.5]*10 + [0]*(len(kpx)-25))
rho

0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
     ... 
67    0.0
68    0.0
69    0.0
70    0.0
71    0.0
Length: 72, dtype: float64

In [134]:
# la formula de euivalencia actuarial

prima_inicial = prima_unica/ sum(rho*discount_factors*kpx)
prima_inicial


6912.092531175566

In [135]:
prima_inicial2 = sum(benefits*discount_factors*kpx)/(sum(rho*discount_factors*kpx))
prima_inicial2 

6912.092531175566

In [136]:
# The annual premiums 
prima_inicial * rho

0     6912.092531
1     6912.092531
2     6912.092531
3     6912.092531
4     6912.092531
         ...     
67       0.000000
68       0.000000
69       0.000000
70       0.000000
71       0.000000
Length: 72, dtype: float64

In [126]:
 # Esperanza de vida reducida de (40)
sum(kpx)-1

38.76660193475971

In [127]:
# EPV de los beneficios si vive hasta los 75
sum(benefits[:36]*discount_factors[:36])

81930.20311867318

In [133]:
# EPV de los beneficios si vive hasta los 95
sum(benefits[:56]*discount_factors[:56])

177099.9393195888

#### <span style='color:midnightblue'>  Edwin Fernandez - BLID
<span style='color:midnightblue'> Python para Actuarios Vida  </span>
***